In [ ]:
from models.DistMult import DistMult
from models.Complex import Complex
from models.ConvE import ConvE, ConvE_args

from utils.loaders import load_data, get_onehots
from utils.evaluation_metrics import SRR, auprc_auroc_ap

import torch
import numpy as np
from sklearn.utils import shuffle

In [ ]:
# split your data 
# train_test_eval_split = 

In [ ]:
def main(model, optimiser, test_data, epochs, batches):
    for epoch in range(epochs):
            #training stage 
            model.train()
            objects, subjects, relationships  = load_data(test_data, batches)

            for index in range(batches):

                obj = torch.LongTensor(objects[index])
                rel = torch.LongTensor(relationships[index])
                subj = torch.squeeze(torch.LongTensor(subjects[index]))

                optimiser.zero_grad()
                pred = model.forward(e1 = obj, rel = rel)
                loss = model.loss(pred, subj)
                loss.backward()
                optimiser.step()


            #evaluation stage
            model.eval()
            objects, subjects, relationships  = load_data(test_data, batches)
            total_sum_reciporical_rank = torch.zeros(1)

            for index in range(batches):

                obj = torch.LongTensor(objects[index])
                rel = torch.LongTensor(relationships[index])
                targets = torch.LongTensor(subjects[index])

                predictions = model.forward(e1 = obj, rel = rel)
                srr = SRR(predictions, targets) 
                total_sum_reciporical_rank = total_sum_reciporical_rank + srr

            print('mean reciporical rank is...')
            print(total_sum_reciporical_rank / len(test_data))



In [ ]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary = get_files()
entities = int(len(lookup)/2)

In [ ]:
batches = 5
epochs = 20
x = shuffle(data)
test_data = x[:50] #just limit data to the first n rows

In [ ]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
all_relationships = torch.LongTensor(all_relationships)

In [ ]:
args = ConvE_args()

In [ ]:
# model = DistMult(num_entities = entities, embedding_dim=100, num_relations=4)
# model = Complex(num_entities = entities, embedding_dim=100, num_relations=4)
model = DistMult(args = args, embedding_dim=200, num_entities=entities, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
main(model=model, optimiser=optimiser, test_data=test_data, epochs=epochs, batches=batches)

In [ ]:
one_hots = get_onehots(targets, entities)
auprc, auroc, ap = auprc_auroc_ap(one_hots, predictions)

In [ ]:
print(auroc)
print(auprc)
print(ap)